<a href="https://colab.research.google.com/github/mahimna/625-714-Project/blob/main/625_714_Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 316 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
!pip install pandas-bokeh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import pandas_bokeh
from scipy.stats import norm
import scipy.linalg as linalg
from datetime import date
from bokeh.models import NumeralTickFormatter, FixedTicker
from bokeh.plotting import figure, show, output_notebook

pandas_bokeh.output_notebook()

In [ ]:
def american_binomial_tree(K, T, S0, r, N, u, d, opttype='P'):
  dt = T/N
  p = (np.exp(r*dt)-d)/(u-d)
  disc = np.exp(-r*dt)
  stock_values = [[0 for i in range(N+1)] for j in range(N+1)]
  for i in range(N+1):
    for j in range(i+1):
      stock_values[i][j] =  S0 * u**j * d**(i-j)
  
  option_values = [[0 for i in range(N+1)] for j in range(N+1)]
  for j in range(N+1):
    option_values[N][j] = max(0, K-stock_values[N][j])

  for i in np.arange(N-1,-1,-1):
    for j in range(i+1):
      continuing_value =  disc * ( p*option_values[i+1][j+1] + (1-p)*option_values[i+1][j] )
      option_values[i][j] = max(continuing_value, K - stock_values[i][j])
  
  return option_values[0][0]

In [ ]:
def generate_gbm_path(initial_price, drift, volatility, num_steps, T):
  prices = []
  current_price = initial_price
  prices.append(initial_price)
  dt = T / num_steps
  while(T - dt > 0):
    dWt = np.random.normal(0, np.sqrt(dt))
    dYt = drift*dt + volatility*dWt
    current_price = current_price + current_price*dYt
    prices.append(current_price)
    T -= dt
  return prices

In [ ]:
def american_simulation(S, K, num_steps, paths, r, T):
  dt = T/num_steps;
  df = np.exp(-r * dt)
  exercise_payoff = [[0 for j in range(num_steps+1)] for i in range(paths)]
  for i in range(paths):
    for j in range(num_steps+1):
      exercise_payoff[i][j] = max(0, K-S[i][j])
  
  cf = [[0 for j in range(num_steps+1)] for i in range(paths)]
  
  for i in range(paths):
    cf[i][num_steps] = exercise_payoff[i][num_steps]

  for j in np.arange(num_steps-1,-1,-1):
    x_vals = []
    y_vals = []
    indices = []
    for i in range(paths):
      if (K-S[i][j]) > 0:
        indices.append(i)
        x_vals.append(S[i][j])
        y = 0
        for k in range(j+1,num_steps+1):
          if cf[i][k] > 0:
            y = cf[i][k]*np.exp(-dt*r*(k-j))
            break
        y_vals.append(y)

    if len(x_vals) > 0:
      poly_fit = np.polyfit(x_vals, y_vals, 2)
      poly_fit_evaluted = np.polyval(poly_fit, x_vals)

      for i in range(len(x_vals)):
        ind = indices[i]
        if (K-S[ind][j]) > poly_fit_evaluted[i]:
          cf[ind][j] = K-S[ind][j]
          for k in range(j+1,num_steps+1):
            cf[ind][k] = 0

  sum_dcf = 0
  for i in range(paths):
    for j in range(num_steps+1):
      sum_dcf += cf[i][j]*np.exp(-dt*r*j)

  return sum_dcf/paths

In [ ]:
def american_finite_difference(S0, K, r, T, sigma, Smax, M, N, is_call=True):
  ds = Smax / float(M)
  dt = T / float(N)
  iValues = np.arange(M)
  jValues = np.arange(N)
  grid = np.zeros(shape=(M,N))
  SValues = np.linspace(0, Smax, M)
  alpha = 0.5*dt*(r*iValues - sigma**2 * iValues**2)
  beta = dt*(r + sigma**2 * iValues**2)
  gamma = -0.5*dt*(r*iValues + sigma**2 * iValues**2)
  coeffs = np.diag(alpha[1:], -1) + np.diag(1 + beta) + np.diag(gamma[:-1],1)

  if is_call:
    grid[:,-1] = np.maximum(SValues - K, 0)
  else:
    grid[:,-1] = np.maximum(K - SValues, 0)
  
  coeffs[0,   0] += 2*alpha[0]
  coeffs[0,   1] -= alpha[0]
  coeffs[-1, -1] += 2*gamma[-1]
  coeffs[-1, -2] -= gamma[-1]
  IV = grid[np.arange(1,M), -1]

  for j in reversed(jValues[:-1]):
    grid[:, j] = linalg.solve(coeffs, grid[:, j+1])

  return np.interp(S0, SValues, grid[:, 0])

In [ ]:
amzn = yf.Ticker("AMZN")

In [ ]:
amzn.options

('2022-08-19',
 '2022-08-26',
 '2022-09-02',
 '2022-09-09',
 '2022-09-16',
 '2022-09-23',
 '2022-10-21',
 '2022-11-18',
 '2022-12-16',
 '2023-01-20',
 '2023-02-17',
 '2023-03-17',
 '2023-06-16',
 '2023-07-21',
 '2023-09-15',
 '2024-01-19',
 '2024-06-21')

In [ ]:
opt = amzn.option_chain('2022-10-21')

In [ ]:
opt.puts.iloc[0]

contractSymbol             AMZN221021P00052000
lastTradeDate        2022-08-05 19:28:01+00:00
strike                                    52.0
lastPrice                                 0.02
bid                                        0.0
ask                                       0.05
change                                   -0.02
percentChange                            -50.0
volume                                     2.0
openInterest                            1796.0
impliedVolatility                     0.781252
inTheMoney                               False
contractSize                           REGULAR
currency                                   USD
Name: 0, dtype: object

In [ ]:
amzn.info['currentPrice']

143.55

In [ ]:
opt.puts[opt.puts['strike']==130]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
54,AMZN221021P00130000,2022-08-05 19:58:23+00:00,130.0,4.62,4.55,4.65,0.22,4.999995,1419.0,7103.0,0.365119,False,REGULAR,USD


In [ ]:
option = opt.puts[opt.puts['strike']==130]

In [ ]:
float(option.impliedVolatility)

0.36511865356445317

In [ ]:
option.lastPrice

54    4.62
Name: lastPrice, dtype: float64

In [ ]:
d0 = date(2022, 10, 21)
d1 = date.today()
delta = d0 - d1

In [ ]:
S0 = amzn.info['currentPrice']      # initial stock price
K = 130               # strike price
T = delta.days/365    # time to maturity in years
r = 0.0244            # annual risk-free rate
vol = float(option.impliedVolatility)

NameError: ignored

In [ ]:
N = 1000
binomial_tree_prices = []
for i in range(1, N):  
  u = np.exp(vol*np.sqrt(T/i)) # up-factor in binomial models
  d = 1/u          # down factor in 
  opttype = 'P' # Option Type 'C' or 'P'
  binomial_tree_prices.append(american_binomial_tree(K,T,S0,r,i,u,d,opttype='P'))

binomial_tree_prices_df = pd.DataFrame(binomial_tree_prices, columns=['Binomial Tree Prices'])

In [ ]:
binomial_tree_prices_df.plot_bokeh(kind='line')

Figure(id='1118', ...)

In [ ]:
N = 1000 
u = np.exp(vol*np.sqrt(T/N)) # up-factor in binomial models
d = 1/u          # down factor in 
opttype = 'P' # Option Type 'C' or 'P'
print(american_binomial_tree(K,T,S0,r,N,u,d,opttype='P'))

4.394266144776024


In [ ]:
M = 500
N = 1000
simulation_prices = []
price_paths = []

for i in range(1, M):
  for j in range(i):
    prices = generate_gbm_path(S0, r, vol, N, T)
    price_paths.append(prices)

  value = american_simulation(S=price_paths, K=K, paths=i, r=r, T=T, num_steps=N-1)
  simulation_prices.append(value)

simulation_prices_df = pd.DataFrame(simulation_prices, columns=['Monte Carlo Simulation Prices'])

In [ ]:
simulation_prices_df.plot_bokeh(kind='line')

Figure(id='1002', ...)

In [ ]:
simulation_prices[-1]

4.9944595999187165

In [ ]:
finite_differences_prices_df.plot_bokeh(kind='line')

Figure(id='2024', ...)

In [ ]:
finite_differences_prices[-1]

4.364872363993067

In [ ]:
finite_differences_prices = []
simulation_prices = []
strikes = []
binomial_tree_prices = []
N = 500

len_puts = len(opt.puts)
for i in range(len_puts):
  option = opt.puts.iloc[i]
  S0 = amzn.info['currentPrice']      # initial stock price
  K = option.strike      # strike price
  T = delta.days/365    # time to maturity in years
  r = 0.0252            # annual risk-free rate
  vol = float(option.impliedVolatility)

  strikes.append(K)

  # Binomial Model Prices
  u = np.exp(vol*np.sqrt(T/N)) # up-factor in binomial models
  d = 1/u          # down factor in 
  opttype = 'P' # Option Type 'C' or 'P'
  binomial_tree_prices.append(american_binomial_tree(K,T,S0,r,N,u,d,opttype='P'))

  # Simulation Prices
  price_paths = []

  for i in range(N):
    prices = generate_gbm_path(S0, r, vol, N, T)
    price_paths.append(prices)

  simulation_prices.append(american_simulation(S=price_paths, K=K, paths=N, r=r, T=T, num_steps=N-1))

  # Finite Difference Prices

  finite_differences_prices.append(american_finite_difference(S0, K, r, T, vol, K*2, N, N, is_call=False))

In [68]:
last_prices = []
len_puts = len(opt.puts)
for i in range(len_puts):
  last_prices.append((opt.puts.iloc[i].bid+opt.puts.iloc[i].ask)/2)

In [69]:
algo_prices = pd.DataFrame(
    {'strikes': strikes[0:135],
     'last_prices': last_prices[0:135],
     'binomial_tree_prices': binomial_tree_prices[0:135],
     'simulation_prices': simulation_prices[0:135],
     'finite_differences_prices': finite_differences_prices[0:135]
    });

In [70]:
algo_prices.plot_bokeh(kind='line',x='strikes')

Figure(id='1789', ...)

In [75]:
binomial_errors = 0
simulation_errors = 0
finite_diffs_errors = 0
for i in range(135):
  binomial_errors += (binomial_tree_prices[i]-last_prices[i])**2
  simulation_errors += (simulation_prices[i]-last_prices[i])**2
  finite_diffs_errors += (finite_differences_prices[i]-last_prices[i])**2

binomial_errors /= 135
simulation_errors /= 135
finite_diffs_errors /= 135 

In [76]:
binomial_errors

0.1263106252281795

In [77]:
simulation_errors

1.7773920937100587

In [78]:
finite_diffs_errors

0.5784209163147855

In [67]:
opt.puts.iloc[117]

contractSymbol             AMZN221021P00161000
lastTradeDate        2022-07-26 19:19:06+00:00
strike                                   161.0
lastPrice                                 46.1
bid                                      18.95
ask                                      20.15
change                                     0.0
percentChange                              0.0
volume                                    51.0
openInterest                             753.0
impliedVolatility                     0.334296
inTheMoney                                True
contractSize                           REGULAR
currency                                   USD
Name: 117, dtype: object